In [ ]:
from PIL import Image
import dask.array as da
from matplotlib import colormaps as cm
import os

In [ ]:
"""
bacteroa shape (t,c,z,y,x): [73, 1, 66, 3796, 3796],
hela cell shape (t,c,z,y,x): [73, 1, 66, 3796, 3796],
organoid shape (t,c,z,y,x): [289, 1, 138, 1838, 1838],
embryo shape (t,c,z,y,x): [161, 1, 116, 6440, 6440],
tissue shape (t,c,z,y,x): [1, 9, 80, 10807, 9890],
"""

dataset_path = {
    'bacteria': '../data/bacteria.ome.zarr/0',
    'hela': '../data/hela.ome.zarr/0',
    'organoid': '../data/organoid.ome.zarr/0',
    'embryo': '../data/embryo.ome.zarr/0',
    'tissue-on': '../data/birefringent-data.ome.zarr/0',
    'tissue-off': '../data/birefringent-data.ome.zarr/0'
}
tcz_slice_axis = [(0, 0, 31), (36, 0, 32), (144, 0, 10), (80, 0, 55), (0, 0, 40), (0, 1, 40)]
xy_crop_regions = [((None, None), (None, None)), ((None, None), (None, None)), ((None, None), (None, None)), ((5,-5),(5,-5)), ((540,-540),(200,-200)), ((540,-540),(200,-200))]
cranges = [(1.33, 1.39), (1.33, 1.39), (1.33, 1.38), (1.33, 1.36), (1.48,1.52), (-0.001,0.001)]
cmap_list = ['gray', 'gray', 'gray', 'gray', 'gray', 'PiYG']

In [ ]:
def array_to_img(image,cmap_name,vmin,vmax):
    norm_img = (image-vmin)/(vmax-vmin)
    color_img = cm.get_cmap(cmap_name)(norm_img)
    color_img = color_img[:,:,:-1]*255
    return Image.fromarray(color_img.astype('u1'), mode='RGB')

In [ ]:
for (data_key,data_path), (t,c,z), (y_crop, x_crop), crange, cmap_name in zip(dataset_path.items(), tcz_slice_axis, xy_crop_regions, cranges, cmap_list):
    data = da.from_zarr(data_path, mode='r')[t,c,:,slice(*y_crop),slice(*x_crop)]
    slice_dataset = {
        'zupper-slice': data[0,:,:],
        'zmiddle-slice': data[z,:,:],
        'zlower-slice': data[-1,:,:],
        'yupper-slice': data[:,0,:],
        'ymiddle-slice': data[:,data.shape[-2]//2,:],
        'ylower-slice': data[:,-1,:],
        'xupper-slice': data[:,:,0],
        'xmiddle-slice': data[:,:,data.shape[-1]//2],
        'xlower-slice': data[:,:,-1],
    }
    for slice_key, slice_data in slice_dataset.items():
        save_path = f'../figure/slice-image/{data_key}-{slice_key}.png'
        if os.path.exists(save_path):
            continue
        result = array_to_img(slice_data.compute(), cmap_name, *crange)
        result.save(save_path)